In [47]:
import argparse
import yaml
import time
import datetime
import cv2
import torch
from torch.autograd import Variable
import numpy as np
from astropy.visualization import make_lupton_rgb
import random
import numpy as np
from scipy.ndimage import zoom
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from scipy import ndimage, misc
import warnings
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import sys


In [48]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [49]:
!cp -r "/content/drive/My Drive/BerkeleyResults/satellite" ./satellite

In [50]:
!ls satellite/

38.820636,-77.01538.png  38.898014,-76.97927.png  38.938009,-76.98513.png
38.831034,-77.00539.png  38.906416,-76.94851.png  38.939464,-77.05280.png
38.864641,-76.98592.png  38.906416,-76.98451.png  38.945614,-77.01564.png
38.8731,-76.97281.png	 38.907205,-77.05396.png  38.952145,-77.07169.png
38.874409,-76.95854.png  38.907725,-77.04406.png  38.954471,-77.06362.png
38.874658,-76.98730.png  38.913193,-76.99032.png  38.95682,-77.05096.png
38.875372,-77.03315.png  38.914004,-77.05731.png  38.956854,-77.05108.png
38.884384,-76.93195.png  38.914656,-77.09722.png  38.971088,-77.02990.png
38.887646,-77.04767.png  38.920896,-76.96704.png  38.972853,-77.05300.png
38.887771,-77.04232.png  38.921453,-77.01448.png  38.972854,-77.05297.png
38.888979,-77.00092.png  38.927132,-77.04890.png  38.980315,-77.05194.png
38.890913,-77.01556.png  38.931573,-77.04472.png  38.985287,-77.03816.png
38.891675,-77.01952.png  38.937768,-77.09681.png  satellite
38.891721,-77.02219.png  38.938009,-76.95813.png


In [51]:
!nvidia-smi

Wed Jun 24 06:32:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    33W / 250W |  13015MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [52]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT', 'DIFF_DAYS']

weather_columns = ['sunriseTime', 'sunsetTime', 'moonPhase', 
                     'precipIntensity', 'precipIntensityMax', 'precipProbability', 'temperatureHigh', 'temperatureHighTime',
                     'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime',
                     'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 
                     'windGustTime', 'windBearing', 'cloudCover', 'uvIndex', 'uvIndexTime','visibility', 'temperatureMin',
                     'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin','apparentTemperatureMinTime', 'apparentTemperatureMax',
                     'apparentTemperatureMaxTime','time']

for item in weather_columns:
    numerical_columns.append(item + str(14))

In [53]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/BerkeleyResults/Approach4/aedes_original_mosquito_weather_semi_clean.csv')
sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week
sentinel_frame["LATITUDE2"]= sentinel_frame["LATITUDE"]
sentinel_frame["LONGITUDE2"]= sentinel_frame["LONGITUDE"]

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category')

categorical_column_sizes = [len(sentinel_frame[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())
    
def get_categorical_tensor(in_frame, categories):
    categorical_data = np.stack([in_frame[col].cat.codes.values for col in categories], 1)
    categorical_data = torch.tensor(categorical_data, dtype=torch.int64)
    return categorical_data

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

count = 0
for i in sentinel_frame.columns:
    print(count, i)
    count+= 1

0 Unnamed: 0
1 X
2 Y
3 OBJECTID
4 TRAPTYPE
5 ATTRACTANTSUSED
6 TRAPID
7 LATITUDE
8 LONGITUDE
9 ADDRESS
10 TOWN
11 STATE
12 COUNTY
13 TRAPSITE
14 TRAPSET
15 SETTIMEOFDAY
16 TRAPCOLLECT
17 YEAR
18 COLLECTTIMEOFDAY
19 GENUS
20 SPECIES
21 TOTAL
22 sunriseTime1
23 sunsetTime1
24 moonPhase1
25 precipIntensity1
26 precipIntensityMax1
27 precipProbability1
28 temperatureHigh1
29 temperatureHighTime1
30 temperatureLow1
31 temperatureLowTime1
32 apparentTemperatureHigh1
33 apparentTemperatureHighTime1
34 apparentTemperatureLow1
35 apparentTemperatureLowTime1
36 dewPoint1
37 humidity1
38 pressure1
39 windSpeed1
40 windGust1
41 windGustTime1
42 windBearing1
43 cloudCover1
44 uvIndex1
45 uvIndexTime1
46 visibility1
47 temperatureMin1
48 temperatureMinTime1
49 temperatureMax1
50 temperatureMaxTime1
51 apparentTemperatureMin1
52 apparentTemperatureMinTime1
53 apparentTemperatureMax1
54 apparentTemperatureMaxTime1
55 icon1
56 time1
57 precipIntensityMaxTime1
58 precipType1
59 summary1
60 sunriseTime2


In [55]:
def paddedzoom(img, zoomfactor=0.8):
    out  = np.zeros_like(img)
    zoomed = cv2.resize(img, None, fx=zoomfactor, fy=zoomfactor)
    
    h, w, _ = img.shape
    zh, zw, _ = zoomed.shape
    if zoomfactor<1:    
        out[int((h-zh)/2):int(-(h-zh)/2),int((w-zw)/2):int(-(w-zw)/2),:] = zoomed
    else:
        out = zoomed[int((zh-h)/2):int(-(zh-h)/2), int((zw-w)/2):int(-(zw-w)/2), :]

    return out

def get_random_crop(image, crop_height, crop_width):

    original_shape = image.shape
    max_x = image.shape[1] - crop_width
    max_y = image.shape[0] - crop_height

    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)

    crop = image[y: y + crop_height, x: x + crop_width]
    resized = cv2.resize(crop, (416, 416), interpolation = cv2.INTER_AREA)

    return resized

def get_affeine(img, rows, cols):
    offset1 = ((random.random() * 2 ) - 1) * 40
    offset2 = ((random.random() * 2 ) - 1) * 40
    M = np.float32([[1,0,offset1],[0,1,offset2]])
    dst = cv2.warpAffine(img,M,(cols,rows))

    return dst

def distort(img, orientation='horizontal', func=np.sin, x_scale=0.05, y_scale=5):
    assert orientation[:3] in ['hor', 'ver'], "dist_orient should be 'horizontal'|'vertical'"
    assert func in [np.sin, np.cos], "supported functions are np.sin and np.cos"
    assert 0.00 <= x_scale <= 0.1, "x_scale should be in [0.0, 0.1]"
    assert 0 <= y_scale <= min(img.shape[0], img.shape[1]), "y_scale should be less then image size"
    img_dist = img.copy()
    
    def shift(x):
        return int(y_scale * func(np.pi * x * x_scale))
    
    for c in range(3):
        for i in range(img.shape[orientation.startswith('ver')]):
            if orientation.startswith('ver'):
                img_dist[:, i, c] = np.roll(img[:, i, c], shift(i))
            else:
                img_dist[i, :, c] = np.roll(img[i, :, c], shift(i))
            
    return img_dist

class SentinelDataset(Dataset):

    def __init__(self, root_dir, data, transform=None):
        self.data = data
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        

        filename = str(self.data.iloc[idx, 555])[0:9]+',' + str(self.data.iloc[idx, 556])[0:9] + '.png'
        img_name = os.path.join(self.root_dir, filename)
        
        image = cv2.imread(img_name)
        image = cv2.resize(image, (416, 416), interpolation = cv2.INTER_AREA)

        #random flip
        if np.random.rand() > 0.5:
            image = np.flip(image, axis=1).copy()
        
        # if np.random.rand() > 0.5:
        #     image = distort(image, orientation=random.choice(['ver','hor']), x_scale=random.uniform(0.01, 0.03), y_scale=random.randint(2,10))
        
        #random rotate
        image = ndimage.rotate(image, random.randint(0,359), axes = [0,1],reshape=False)

        # random zoom
        or_image = image.copy()
        image = paddedzoom(image, 1.0 + (np.random.rand()/2) )
        if image.shape != or_image.shape:
            image = or_image
        
        if np.random.rand() > 0.2:
            image = get_random_crop(image, 380, 380)
        
        if np.random.rand() > 0.5:
            image = get_affeine(image, 416, 416)

        image = torch.from_numpy(image)

        current_row = self.data.iloc[[idx]]
        target = current_row['TOTAL'].values

        category_tensor = get_categorical_tensor(current_row, categorical_columns)
        numerical_tensor = get_numerical_tensor(current_row, numerical_columns)

        # from matplotlib.pyplot import figure
        # from matplotlib import pyplot as PLT


        # PLT.imshow(image)
        # PLT.show()
        # print(numerical_tensor)
        # print(target)
        # print(idx)
        # raise ValueError('A very specific bad thing happened.')

        return image,category_tensor ,numerical_tensor, target

In [56]:
import torch
import torch.nn as nn

from collections import defaultdict

class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

def add_conv(in_ch, out_ch, ksize, stride):
    stage = nn.Sequential()
    pad = (ksize - 1) // 2
    stage.add_module('conv', nn.Conv2d(in_channels=in_ch,
                                       out_channels=out_ch, kernel_size=ksize, stride=stride,
                                       padding=pad, bias=False))
    stage.add_module('batch_norm', nn.BatchNorm2d(out_ch))
    stage.add_module('leaky', nn.LeakyReLU(0.1))
    return stage

def add_linear(in_ch, in_dm, out_ch, leaky = True):
    stage = nn.Sequential()
    stage.add_module('linear', nn.Linear(in_features=in_ch*in_dm*in_dm, out_features=out_ch))
    stage.add_module('batch_norm',nn.BatchNorm1d(num_features=out_ch))
    if leaky :
        stage.add_module('LeakyRelu', nn.LeakyReLU())
    else:
        stage.add_module('relu', nn.ReLU())

    return stage


class resblock(nn.Module):
    def __init__(self, ch, nblocks=1, shortcut=True):

        super().__init__()
        self.shortcut = shortcut
        self.module_list = nn.ModuleList()
        for i in range(nblocks):
            resblock_one = nn.ModuleList()
            resblock_one.append(add_conv(ch, ch//2, 1, 1))
            resblock_one.append(add_conv(ch//2, ch, 3, 1))
            self.module_list.append(resblock_one)

    def forward(self, x):
        for module in self.module_list:
            h = x
            for res in module:
                h = res(h)
            x = x + h if self.shortcut else h
        return x

def create_darknet_modules():

    # DarkNet53
    mlist = nn.ModuleList()
    mlist.append(add_conv(in_ch=3, out_ch=32, ksize=3, stride=1))
    mlist.append(add_conv(in_ch=32, out_ch=64, ksize=3, stride=2))
    mlist.append(resblock(ch=64))
    mlist.append(add_conv(in_ch=64, out_ch=128, ksize=3, stride=2))
    mlist.append(resblock(ch=128, nblocks=2))
    mlist.append(add_conv(in_ch=128, out_ch=256, ksize=3, stride=2))
    mlist.append(resblock(ch=256, nblocks=8))
    mlist.append(add_conv(in_ch=256, out_ch=512, ksize=3, stride=2))
    mlist.append(resblock(ch=512, nblocks=8)) 
    mlist.append(add_conv(in_ch=512, out_ch=1024, ksize=3, stride=2))
    mlist.append(resblock(ch=1024, nblocks=4))

    mlist.append(Flatten())
    mlist.append(add_linear(in_ch=1024, in_dm=13, out_ch=2048))
    mlist.append(add_linear(in_ch=2048, in_dm=1, out_ch=1024))
    #mlist.append(add_linear(in_ch=1024, in_dm=1, out_ch=1, leaky = False))

    return mlist

def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout = 0.3):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols

    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.Dropout(dropout))

    # all_layers.append(nn.Linear(hidden_size, output_size))
    # all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

def create_mix_modules(input_size, output_size, dropout = 0.3):
    all_layers = []
    all_layers.append(nn.Linear(input_size, input_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(input_size, input_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(input_size, output_size))
    all_layers.append(nn.ReLU())
    return nn.Sequential(*all_layers)

def turn_conv_lstm_autograd(self):

    def dfs_off(model):
        for name, child in model.named_children():
            for param in child.parameters():
                param.requires_grad = False
            dfs_off(child)

    for i, module in enumerate(self.module_list):
        if i == 11:
          break
        dfs_off(module)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size):
        super(Sentinel_net, self).__init__()
        self.module_list = create_darknet_modules()
        turn_conv_lstm_autograd(self)
    
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, 65)
        self.mix_layers = create_mix_modules(1089, 1)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(0.3)

    def forward(self, x, x_categorical, x_numerical):
        for i, module in enumerate(self.module_list):
            x = module(x)

        x = x.squeeze(dim=0)

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        
        y = torch.cat(embeddings, 1)

        y = self.embedding_dropout(y)
        y = torch.cat([y, x_numerical], 1)

        y = self.layers(y).squeeze()
                  
        #print('x ', x.shape,'y ', y.shape)

        mixtensor = torch.cat((x, y), 1)
        mixtensor = self.mix_layers(mixtensor)
        return mixtensor

In [57]:
from __future__ import division
import torch
import numpy as np


def parse_conv_block(m, weights, offset, initflag):
    """
    Initialization of conv layers with batchnorm
    Args:
        m (Sequential): sequence of layers
        weights (numpy.ndarray): pretrained weights data
        offset (int): current position in the weights file
        initflag (bool): if True, the layers are not covered by the weights file. \
            They are initialized using darknet-style initialization.
    Returns:
        offset (int): current position in the weights file
        weights (numpy.ndarray): pretrained weights data
    """
    conv_model = m[0]
    bn_model = m[1]
    param_length = m[1].bias.numel()

    # batchnorm
    for pname in ['bias', 'weight', 'running_mean', 'running_var']:
        layerparam = getattr(bn_model, pname)

        if initflag: # yolo initialization - scale to one, bias to zero
            if pname == 'weight':
                weights = np.append(weights, np.ones(param_length))
            else:
                weights = np.append(weights, np.zeros(param_length))

        param = torch.from_numpy(weights[offset:offset + param_length]).view_as(layerparam)
        layerparam.data.copy_(param)
        offset += param_length

    param_length = conv_model.weight.numel()

    # conv
    if initflag: # yolo initialization
        n, c, k, _ = conv_model.weight.shape
        scale = np.sqrt(2 / (k * k * c))
        weights = np.append(weights, scale * np.random.normal(size=param_length))

    param = torch.from_numpy(
        weights[offset:offset + param_length]).view_as(conv_model.weight)
    conv_model.weight.data.copy_(param)
    offset += param_length

    return offset, weights

def parse_yolo_block(m, weights, offset, initflag):
    """
    YOLO Layer (one conv with bias) Initialization
    Args:
        m (Sequential): sequence of layers
        weights (numpy.ndarray): pretrained weights data
        offset (int): current position in the weights file
        initflag (bool): if True, the layers are not covered by the weights file. \
            They are initialized using darknet-style initialization.
    Returns:
        offset (int): current position in the weights file
        weights (numpy.ndarray): pretrained weights data
    """
    conv_model = m._modules['conv']
    param_length = conv_model.bias.numel()

    if initflag: # yolo initialization - bias to zero
        weights = np.append(weights, np.zeros(param_length))

    param = torch.from_numpy(
        weights[offset:offset + param_length]).view_as(conv_model.bias)
    conv_model.bias.data.copy_(param)
    offset += param_length

    param_length = conv_model.weight.numel()

    if initflag: # yolo initialization
        n, c, k, _ = conv_model.weight.shape
        scale = np.sqrt(2 / (k * k * c))
        weights = np.append(weights, scale * np.random.normal(size=param_length))
 
    param = torch.from_numpy(
        weights[offset:offset + param_length]).view_as(conv_model.weight)
    conv_model.weight.data.copy_(param)
    offset += param_length

    return offset, weights

def parse_yolo_weights(model, weights_path):
    """
    Parse YOLO (darknet) pre-trained weights data onto the pytorch model
    Args:
        model : pytorch model object
        weights_path (str): path to the YOLO (darknet) pre-trained weights file
    """
    fp = open(weights_path, "rb")

    # skip the header
    header = np.fromfile(fp, dtype=np.int32, count=5) # not used
    # read weights 
    weights = np.fromfile(fp, dtype=np.float32)
    fp.close()

    offset = 0 
    initflag = False #whole yolo weights : False, darknet weights : True

    for m in model.module_list:
        if m._get_name() == 'Sequential':
            # normal conv block
            offset, weights = parse_conv_block(m, weights, offset, initflag)

        elif m._get_name() == 'resblock':
            # residual block
            for modu in m._modules['module_list']:
                for blk in modu:
                    offset, weights = parse_conv_block(blk, weights, offset, initflag)
        else:
            break
        # elif m._get_name() == 'YOLOLayer':
        #     # YOLO Layer (one conv with bias) Initialization
        #     offset, weights = parse_yolo_block(m, weights, offset, initflag)

        # initflag = (offset >= len(weights)) # the end of the weights file. turn the flag on

In [58]:
def train_darknet_net(sentinel_dataset_train,sentinel_dataset_test, fold_count):
    model = Sentinel_net(categorical_embedding_sizes,len(numerical_columns),1)
    parse_yolo_weights(model, "/content/drive/My Drive/Colab/mosquito/darknet53.conv.74")
    model = model.cuda()
    model.train()

    # criterion = torch.nn.MSELoss(size_average = True) 
    criterion = torch.nn.L1Loss()

    params_dict = dict(model.named_parameters())
    params = []
    base_lr = 0.0001

    for key, value in params_dict.items():
        if value.requires_grad:
          print(key)
          if 'weight' in key:
              params += [{'params':value, 'weight_decay':4e-3}]
          else:
              params += [{'params':value, 'weight_decay':0.0}]

    optimizer = torch.optim.RMSprop(params, lr=base_lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9, centered=False)

    epochs = 20
    min_eval = sys.maxsize
    current_score = 0

    for epoch in range(0,epochs):
        loss_sum = 0
        
        model.train()
        for i_batch, sample_batched in enumerate(train_dataset_loader):
            image = sample_batched[0].cuda().float()
            image = image.permute(0,3,2,1)/255
            cat_tensor = sample_batched[1].squeeze().cuda()
            num_tensor = sample_batched[2].squeeze().cuda().float()

            #print('image shape ', image.shape,'cat_tensor shape ', cat_tensor.shape,'num_tensor shape ', num_tensor.shape)

            y_pred = model(image, cat_tensor, num_tensor)
            y = sample_batched[3].cuda().float()

            loss = criterion(y_pred, y) 
            loss.backward() 
            loss_sum += loss.item()
            optimizer.step() 
            optimizer.zero_grad() 

        model.eval()
        print('Train Accuracy epoch {}, loss {} %'.format(epoch, loss_sum))

        with torch.no_grad():

            if epoch % 10 == 0:
                model.eval()
                eval_loss_sum = 0 
                result_pred = []
                result_act = []
                with torch.no_grad():
                    y2 = None
                    y_pred2 = None
                    for i_batch2, sample_batched2 in enumerate(test_dataset_loader):

                        image2 = sample_batched2[0].cuda().float()
                        image2 = image2.permute(0,3,2,1)/255
                        cat_tensor2 = sample_batched2[1].squeeze().cuda()
                        num_tensor2 = sample_batched2[2].squeeze().cuda().float()
                        #print('test shape ', image.shape,'test shape ', cat_tensor.shape,'test shape ', num_tensor.shape)

                        y_pred2 = model(image2,cat_tensor2,num_tensor2)
                        y2 = sample_batched2[3].cuda().float()
                        eval_loss_sum += criterion(y_pred2, y2) 
                        result_pred += y_pred2.tolist()
                        result_act += y2.tolist()
                    
                    print('')
                    print(' Test Accuracy  {} %'.format( eval_loss_sum))


                    if eval_loss_sum < min_eval:
                        current_score = mean_absolute_error(result_pred, result_act)
                        print("Score!" ,current_score )
                        min_eval = eval_loss_sum
                        print("saved!" ,current_score )
                        torch.save({'iter': epoch,
                          'model_state_dict': model.state_dict(),
                          'optimizer_state_dict': optimizer.state_dict(),
                          'best_ac': min_eval,
                          },
                          os.path.join('/content/drive/My Drive/BerkeleyResults/Approach4/aedes_best_approach' + str(fold_count) + '.ckpt'))


    return current_score

In [60]:
start_time = time.time()

scores = []
cv = KFold(n_splits=5, random_state=42, shuffle=True)
fold_count = 0
for train_index, test_index in cv.split(sentinel_frame, None):
    print('Fold')
    train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

    sentinel_dataset_train = SentinelDataset(data=train, root_dir='satellite/')
    sentinel_dataset_test = SentinelDataset(data=test, root_dir='satellite/')

    train_dataset_loader = torch.utils.data.DataLoader(dataset=sentinel_dataset_train,
                                                    batch_size=20,
                                                    shuffle=False)
    
    test_dataset_loader = torch.utils.data.DataLoader(dataset=sentinel_dataset_test,
                                                    batch_size=20,
                                                    shuffle=False)
    

    current_score = train_darknet_net(sentinel_dataset_train,sentinel_dataset_test, fold_count)
    scores.append(current_score)
    fold_count += 1
print(scores)
print( sum(scores) / len(scores) )
print("--- %s seconds ---" % (time.time() - start_time))

Fold
module_list.12.linear.weight
module_list.12.linear.bias
module_list.12.batch_norm.weight
module_list.12.batch_norm.bias
module_list.13.linear.weight
module_list.13.linear.bias
module_list.13.batch_norm.weight
module_list.13.batch_norm.bias
layers.0.weight
layers.0.bias
layers.3.weight
layers.3.bias
layers.6.weight
layers.6.bias
mix_layers.0.weight
mix_layers.0.bias
mix_layers.3.weight
mix_layers.3.bias
mix_layers.6.weight
mix_layers.6.bias
all_embeddings.0.weight
all_embeddings.1.weight
all_embeddings.2.weight
all_embeddings.3.weight
all_embeddings.4.weight
all_embeddings.5.weight
all_embeddings.6.weight
all_embeddings.7.weight
Train Accuracy epoch 0, loss 213.32956504821777 %

 Test Accuracy  49.134647369384766 %
Score! 5.505054664611817
saved! 5.505054664611817
Train Accuracy epoch 1, loss 184.1669783592224 %
Train Accuracy epoch 2, loss 176.52554416656494 %
Train Accuracy epoch 3, loss 176.21478176116943 %
Train Accuracy epoch 4, loss 178.66790795326233 %
Train Accuracy epoch 5